Phabricator: [T266714](https://phabricator.wikimedia.org/T266714)

MVP specification:

- Top 100 pages by traffic from US from English, Chinese, and Spanish Wikipedias over the past 6 months
  - Only those pages for which we have at least one topic predicted with 50% or greater score
  - Increase to 500 pages and 12 months in next iteration after reviewing performance
- Last 6 months of pageviews and edit counts
  - Inrease to 12 months in next iteration after reviewing performance
- 1 table for view counts, split by access method (desktop vs mobile web vs mobile app) and agent type (user vs spider vs automated)
- 1 table for edit counts, split by editor type (registered vs anonymous, registered bot vs human)
- Topic data from `isaacj.article_topics_outlinks_2020_09` and `cchen.topic_component`:
  - `score` must be 0.5 or greater
  - `topics`, `main_topics`, and `sub_topics` (based on [this taxonomy](https://www.mediawiki.org/wiki/ORES/Articletopic))
- Daily granularity

Steps:



In [1]:
suppressPackageStartupMessages({
    library(purrr)
    library(dplyr)
    library(tidyr)
    library(wmfdata)
    library(here)
    library(readr) # read/write_rds
    library(glue)
    library(processx)
})

In [2]:
fs::dir_create("data")

In [3]:
top_viewed_query <- "USE wmf;
SELECT
  project, page_id, page_title,
  SUM(view_count) AS total_views
FROM pageview_hourly
WHERE year = 2020 AND month = ${month}
  AND project = '${project}'
  AND country_code = 'US' -- from United States
  AND access_method IN('desktop', 'mobile web', 'mobile app')
  AND agent_type = 'user' -- non-automated
  AND namespace_id = 0 -- article namespace only
  -- exclude main pages (homepages)
GROUP BY project, page_id, page_title
ORDER BY total_views DESC
LIMIT 1000;"

get_top_viewed <- function() {
    top_viewed_rds <- here("data", "top_viewed.rds")
    # Return cached results if possible or fetch-cache-return:
    if (fs::file_exists(top_viewed_rds)) {
        top_viewed <- read_rds(top_viewed_rds)
    } else {
        top_viewed <- map_dfr(
            c("English" = "en.wikipedia", "Chinese" = "zh.wikipedia", "Spanish" = "es.wikipedia"),
            function(project) {
                monthly <- map_dfr(
                    5:10,
                    function(month) {
                        query <- glue(top_viewed_query, .open = "${")
                        suppressMessages(result <- query_hive(query))
                        result$month <- month
                        return(result)
                    }
                )
            },
            .id = "wikipedia"
        )
        write_rds(top_viewed, top_viewed_rds, compress = "gz") # cache query results
    }
    return(top_viewed)
}

top_viewed <- get_top_viewed()

In [4]:
head(top_viewed)

,wikipedia,project,page_id,page_title,total_views,month
,<chr>,<chr>,<int>,<chr>,<int>,<int>
1,English,en.wikipedia,15580374,Main_Page,68367614,5
2,English,en.wikipedia,909036,Elon_Musk,3376579,5
3,English,en.wikipedia,20455,Michael_Jordan,3339421,5
4,English,en.wikipedia,54803678,Antifa_(United_States),2267750,5
5,English,en.wikipedia,64076090,Death_of_George_Floyd,2033834,5
6,English,en.wikipedia,27419466,Judy_Mikovits,1985023,5


In [5]:
top_viewed <- top_viewed %>%
    mutate(
        is_mainpage = (
            (project == "en.wikipedia" & page_title == "Main_Page")
            | (project == "zh.wikipedia" & page_title == "首页")
            | (project == "es.wikipedia" & page_title == "Portada")
        )
    )

In [6]:
top_viewed %>%
    group_by(project, is_mainpage) %>%
    summarize(pages = length(unique(page_id)), .groups = "drop")

project,is_mainpage,pages
<chr>,<lgl>,<int>
en.wikipedia,FALSE,3016
en.wikipedia,TRUE,1
es.wikipedia,FALSE,2394
zh.wikipedia,FALSE,2370
zh.wikipedia,TRUE,1


In [7]:
top_viewed %>%
    filter(project == "es.wikipedia") %>%
    group_by(page_title) %>%
    summarize(total_views = sum(total_views), .groups = "drop") %>%
    top_n(10, total_views)

page_title,total_views
<chr>,<int>
Descubrimiento_de_América,147452
Deutsche_Welle,165341
Estados_Unidos,169822
Facebook,193269
Google,270578
Independencia_de_México,199228
Tabla_periódica_de_los_elementos,254647
Traducción,446671
Zodiaco,222863


In [8]:
top_X00 <- top_viewed %>%
    group_by(project, is_mainpage, page_id, page_title) %>%
    summarize(total_views = sum(total_views), .groups = "drop") %>%
    group_by(project, is_mainpage) %>%
    top_n(500, total_views) %>% # change 100 to 500 later
    ungroup

In [9]:
top_X00 <- top_X00 %>%
    mutate(
        wiki_db = case_when(
            project == "en.wikipedia" ~ "enwiki",
            project == "zh.wikipedia" ~ "zhwiki",
            project == "es.wikipedia" ~ "eswiki"
        )
    )

In [10]:
top_X00 %>%
    select(project, wiki_db, page_id, page_title) %>%
    write_tsv(here("data/top_x00.tsv"), col_names = FALSE)

In [11]:
run("hive", args = c("-f", "queries/create_top_viewed.hql")) # hive -f queries/create_top_viewed.hql

$status
[1] 0

$stdout
[1] ""

$stderr
[1] "log4j:WARN No such property [maxBackupIndex] in org.apache.log4j.DailyRollingFileAppender.\n\nLogging initialized using configuration in file:/etc/hive/conf.analytics-hadoop/hive-log4j.properties\nOK\nTime taken: 0.963 seconds\nOK\nTime taken: 0.31 seconds\nLoading data to table bearloga.top_viewed\nOK\nTime taken: 0.55 seconds\n"

$timeout
[1] FALSE

To be run on both stat1004 and stat1006:

```
library(processx)
library(glue)
library(purrr)
```

To be run on stat1004:

```
results <- map_dfr(
    5:10,
    function(month) {
        run("hive", args = c("-f", "queries/cda_views.hql", "-d", glue("month={month}")))
    }
)
```

To be run on stat1006:

```
results <- map_dfr(
    5:10,
    function(month) {
        run("hive", args = c("-f", "queries/cda_edits.hql", "-d", "snapshot=2020-10", "-d", glue("month={month}")))
    }
)
```